This script builts 5 different predictive models for an individual for the whole period of available data. The algorithms are based on the results of notebook 2. LinearSVC,AdaBoost Classifier,Logistic Regression, MLPClassifier, and SGDClassifier

In [ ]:
#connect oracle using windows client..
#comment:download correct version:3.5 python+ Oracle 12c  windows 64
#download and pip install cx_Oracle-5.2+oci12c-cp35-none-win_amd64.whl from lfd.uci.edu/~gohlke/pythonlibs/
#cx_oracle 
#commect:Download the right oracle client 12c+ windows 64 and install in C:\instantclient_12_1
#http://www.oracle.com/technetwork/topics/winx64soft-089540.html
#Set path in windows add C:\instantclient_12_1 (use path manager.exe) restart

import cx_Oracle
import pandas as pd
import numpy as np
import time

# classifiers & testing
from sklearn.svm import LinearSVC
from sklearn.svm import SVC
from sklearn.neural_network import MLPClassifier
from sklearn.tree import DecisionTreeClassifier
from sklearn.model_selection import train_test_split, GridSearchCV
from sklearn.metrics import f1_score, confusion_matrix, accuracy_score
from sklearn import metrics
from sklearn.ensemble import RandomForestClassifier
from sklearn.ensemble import ExtraTreesClassifier
from sklearn.preprocessing import Normalizer
from sklearn.neighbors import KNeighborsClassifier
from sklearn.linear_model import LogisticRegression
from sklearn.linear_model import SGDClassifier
from sklearn.model_selection import KFold
from sklearn.ensemble import AdaBoostClassifier
from sklearn import model_selection
from sklearn import preprocessing
from sklearn.naive_bayes import BernoulliNB

import itertools
import re  
import os 
import pickle
from mpl_toolkits.mplot3d import Axes3D
from matplotlib import colors
import matplotlib.pyplot as plt
from matplotlib.colors import ListedColormap
%matplotlib inline
import seaborn as sns
import json
import collections
from math import sqrt
from random import randrange
from random import seed

import sys
import itertools
import warnings
from sklearn.preprocessing import Binarizer, MaxAbsScaler, MinMaxScaler
from sklearn.preprocessing import Normalizer, PolynomialFeatures, RobustScaler, StandardScaler
from sklearn.decomposition import FastICA, PCA
from sklearn.kernel_approximation import RBFSampler, Nystroem
from sklearn.cluster import FeatureAgglomeration
from sklearn.feature_selection import SelectFwe, SelectKBest, SelectPercentile, VarianceThreshold
from sklearn.feature_selection import SelectFromModel, RFE
from sklearn.ensemble import ExtraTreesClassifier
from sklearn.linear_model import SGDClassifier
from sklearn.model_selection import cross_val_predict
from sklearn.metrics import accuracy_score, f1_score
from sklearn.pipeline import make_pipeline
import itertools

con = cx_Oracle.connect('hanzefitbit/hanzefitbit@127.0.0.1/xe')


#con.close()

In [ ]:
#function to determine if people have reached their goal of the day

def daily_steps_cat_f (steps_value,threshold):
    if (steps_value<threshold):
        #print('smaller then threshold')
        return 0
    if (steps_value>=threshold):
        #print('more then threshold')
        return 1
    

In [ ]:
def save_model_f (dest,treatment_id,algorithm_name):
    pickle_model=str(treatment_id)+'_'+algorithm_name+'_'+'model.pkl' 
    #if algorithm_name=='SVC':
    #    pickle.dump(globals()['clf%s' % algorithm_name+treatment_id],open(os.path.join(dest,pickle_model),'wb'),protocol=4)    
    #else:    
    pickle.dump(globals()['clf%s' % algorithm_name+treatment_id].best_estimator_,open(os.path.join(dest,pickle_model),'wb'),protocol=4)

In [ ]:
def pickle_destination_f (root,leaf):
    dest = os.path.join(root,leaf) 
    if not os.path.exists(dest): 
        os.makedirs(dest) 
    return dest    

In [ ]:
def all_same(items):
    return all(x == items[0] for x in items)


In [ ]:
def insert_hyperparameter_into_database_f (model_id,treatment_id,algorithm_name,parameter_name):
    parameter_id=find_parameter_id_f(algorithm_name,parameter_name)
    if parameter_name=='C':
        value=globals()['clf%s' % algorithm_name+treatment_id].best_estimator_.C
        insert_into_model_parameters_f(model_id,treatment_id,parameter_id,value)
    if parameter_name=='max_features':
        value=globals()['clf%s' % algorithm_name+treatment_id].best_estimator_.max_features
        insert_into_model_parameters_f(model_id,treatment_id,parameter_id,value)    
    if parameter_name=='criterion':
        value=globals()['clf%s' % algorithm_name+treatment_id].best_estimator_.criterion
        insert_into_model_parameters_f(model_id,treatment_id,parameter_id,value)
    if parameter_name=='penalty':
        value=globals()['clf%s' % algorithm_name+treatment_id].best_estimator_.penalty
        insert_into_model_parameters_f(model_id,treatment_id,parameter_id,value)
    #if parameter_name=='min_impurity_decrease':
        #value=globals()['clf%s' % algorithm_name+treatment_id].best_estimator_.min_impurity_decrease
        #insert_into_model_parameters_f(model_id,treatment_id,parameter_id,value)    
    if parameter_name=='n_estimators':
        value=globals()['clf%s' % algorithm_name+treatment_id].best_estimator_.n_estimators
        insert_into_model_parameters_f(model_id,treatment_id,parameter_id,value)     
    if parameter_name=='fit_intercept':
        value=globals()['clf%s' % algorithm_name+treatment_id].best_estimator_.fit_intercept
        value=str(value)
        insert_into_model_parameters_f(model_id,treatment_id,parameter_id,value)  
    if parameter_name=='learning_rate':
        value=globals()['clf%s' % algorithm_name+treatment_id].best_estimator_.learning_rate
        insert_into_model_parameters_f(model_id,treatment_id,parameter_id,value)
    if parameter_name=='alpha':
        value=globals()['clf%s' % algorithm_name+treatment_id].best_estimator_.alpha
        insert_into_model_parameters_f(model_id,treatment_id,parameter_id,value)
    if parameter_name=='binarize':
        value=globals()['clf%s' % algorithm_name+treatment_id].best_estimator_.binarize
        insert_into_model_parameters_f(model_id,treatment_id,parameter_id,value)   
    if parameter_name=='fit_prior':
        value=globals()['clf%s' % algorithm_name+treatment_id].best_estimator_.fit_prior
        value=str(value)
        insert_into_model_parameters_f(model_id,treatment_id,parameter_id,value)
    if parameter_name=='class_prior':
        value=globals()['clf%s' % algorithm_name+treatment_id].best_estimator_.class_prior
        value=str(value)
        insert_into_model_parameters_f(model_id,treatment_id,parameter_id,value)
    if parameter_name=='loss':
        value=globals()['clf%s' % algorithm_name+treatment_id].best_estimator_.loss
        insert_into_model_parameters_f(model_id,treatment_id,parameter_id,value)
    if parameter_name=='dual':
        value=globals()['clf%s' % algorithm_name+treatment_id].best_estimator_.dual
        value=str(value)
        insert_into_model_parameters_f(model_id,treatment_id,parameter_id,value)
    if parameter_name=='activation':
        value=globals()['clf%s' % algorithm_name+treatment_id].best_estimator_.activation
        insert_into_model_parameters_f(model_id,treatment_id,parameter_id,value)
    if parameter_name=='learning_rate_init':
        value=globals()['clf%s' % algorithm_name+treatment_id].best_estimator_.learning_rate_init
        insert_into_model_parameters_f(model_id,treatment_id,parameter_id,value)
    if parameter_name=='kernel':
        value=globals()['clf%s' % algorithm_name+treatment_id].best_estimator_.kernel
        insert_into_model_parameters_f(model_id,treatment_id,parameter_id,value)    
    if parameter_name=='l1_ratio':
        value=globals()['clf%s' % algorithm_name+treatment_id].best_estimator_.l1_ratio
        insert_into_model_parameters_f(model_id,treatment_id,parameter_id,value)    
    if parameter_name =='metric':
        value=globals()['clf%s' % algorithm_name+treatment_id].best_estimator_.metric
        insert_into_model_parameters_f(model_id,treatment_id,parameter_id,value)
    if parameter_name =='weights':
        value=globals()['clf%s' % algorithm_name+treatment_id].best_estimator_.weights
        insert_into_model_parameters_f(model_id,treatment_id,parameter_id,value)
    if parameter_name =='n_neighbors':
        value=globals()['clf%s' % algorithm_name+treatment_id].best_estimator_.n_neighbors
        insert_into_model_parameters_f(model_id,treatment_id,parameter_id,value) 
    if parameter_name =='loss':
        value=globals()['clf%s' % algorithm_name+treatment_id].best_estimator_.loss
        insert_into_model_parameters_f(model_id,treatment_id,parameter_id,value)   
        
           
    

In [ ]:
def save_model_database_f(treatment_id,pickle_model,algorithm_name,dest):
    #Save description of models into hft_models_t
        sql = "SELECT HFT_MODEL_T_ID_SEQ.NEXTVAL FROM DUAL"
        cur.execute(sql)
        #cur.fetchone [0] returns value,cur.fetchone returns tuple
        model_id=cur.fetchone()[0]
        sql = "INSERT INTO HFT_MODEL_T(ID,NAME,ALGORITHM,DESTINATION) VALUES (:1,:2,:3,:4)"
        cur.execute(sql,(model_id,pickle_model,algorithm_name,dest))
        con.commit()
        
         #insert the tuned parameter values into the database    
        if algorithm_name=='LR':
            insert_hyperparameter_into_database_f (model_id,treatment_id,algorithm_name,'C')
            insert_hyperparameter_into_database_f (model_id,treatment_id,algorithm_name,'penalty')
            insert_hyperparameter_into_database_f(model_id,treatment_id,algorithm_name,'fit_intercept')
        elif algorithm_name=='ADA':
            insert_hyperparameter_into_database_f (model_id,treatment_id,algorithm_name,'learning_rate')
            insert_hyperparameter_into_database_f (model_id,treatment_id,algorithm_name,'n_estimators')  
        elif algorithm_name =='RF':
            insert_hyperparameter_into_database_f (model_id,treatment_id,algorithm_name,'criterion')
            insert_hyperparameter_into_database_f (model_id,treatment_id,algorithm_name,'max_features') 
            #insert_hyperparameter_into_database_f (model_id,treatment_id,algorithm_name,'min_impurity_decrease')
            insert_hyperparameter_into_database_f (model_id,treatment_id,algorithm_name,'n_estimators') 
        elif algorithm_name=='NN':
            insert_hyperparameter_into_database_f (model_id,treatment_id,algorithm_name,'learning_rate')
            insert_hyperparameter_into_database_f (model_id,treatment_id,algorithm_name,'activation')
            insert_hyperparameter_into_database_f(model_id,treatment_id,algorithm_name,'learning_rate_init')
        elif algorithm_name=='SGD':
            insert_hyperparameter_into_database_f (model_id,treatment_id,algorithm_name,'fit_intercept')
            insert_hyperparameter_into_database_f (model_id,treatment_id,algorithm_name,'l1_ratio')
            insert_hyperparameter_into_database_f (model_id,treatment_id,algorithm_name,'loss')
        elif algorithm_name=='SVC':
            insert_hyperparameter_into_database_f (model_id,treatment_id,algorithm_name,'kernel')
        elif algorithm_name =='DT':
            insert_hyperparameter_into_database_f (model_id,treatment_id,algorithm_name,'criterion')
            insert_hyperparameter_into_database_f (model_id,treatment_id,algorithm_name,'max_features') 
        elif algorithm_name =='KNN':
            insert_hyperparameter_into_database_f (model_id,treatment_id,algorithm_name,'metric')
            insert_hyperparameter_into_database_f (model_id,treatment_id,algorithm_name,'weights') 
            insert_hyperparameter_into_database_f (model_id,treatment_id,algorithm_name,'n_neighbors') 
        
        #save the perfomance of the models into the hft_metrics_t 
        #get values of confusion matrix
        globals()['df_confusion_matrix%s' % treatment_id]=pd.DataFrame(metrics.confusion_matrix(globals()['y%s' % treatment_id], globals()['y_pred%s' % algorithm_name+treatment_id]))
          
        #threshold
        THRESHOLD=int(globals()['df_threshold%s' % treatment_id]['avg_daily_steps'])
       
       
        #true negative
        TN = int(globals()['df_confusion_matrix%s' % treatment_id].iloc[0,0])

               
        #true positive
        FP = int(globals()['df_confusion_matrix%s' % treatment_id].iloc[0,1])

        #false negative
        FN = int(globals()['df_confusion_matrix%s' % treatment_id].iloc[1,0])

        #true positive
        TP = int(globals()['df_confusion_matrix%s' % treatment_id].iloc[1,1])

        #f1 score
        F1 = float(f1_score(globals()['y%s' % treatment_id], globals()['y_pred%s' % algorithm_name+treatment_id], average='macro'))
         
        #Accuracy
        ACC = float(metrics.accuracy_score(globals()['y%s' % treatment_id], globals()['y_pred%s' % algorithm_name+treatment_id]))
        
        #number of observations
              
        OBS=int(len(globals()['df%s' % treatment_id].index))
        
        # -1 is for all weekdays
        WEEKDAY=-1
        #insert results into database
        sql="INSERT INTO HFT_METRICS_T(F1_SCORE,TRUE_NEGATIVE,TRUE_POSITIVE,FALSE_NEGATIVE,FALSE_POSITIVE,HFT_TREATMENT_ID,HFT_MODEL_ID,THRESHOLD,ACCURACY,NUMBER_OF_OBSERVATIONS,WEEKDAY)\
        VALUES (:1,:2,:3,:4,:5,:6,:7,:8,:9,:10,:11)"
        cur.execute(sql,(F1,TN,TP,FN,FP,treatment_id,model_id,THRESHOLD,ACC,OBS,WEEKDAY))
        con.commit()
         
      

In [ ]:
def fit_model (treatment_id,use_algorithm_name,use_algorithm,X_train,y_train):
    treatment_id_name=str(treatment_id)
    
    
    if use_algorithm_name =='LR':
        #use_algorithm = LogisticRegression()
        #parameters GridSearch Logistic Regression
        clf__penalty_param= ['l1', 'l2']
        clf__C_param= [0.001, 0.01, 0.1, 1, 10, 100, 1000] 
        clf_fit_intercept=[True, False]
        parameters = {'C':clf__C_param, 'penalty':clf__penalty_param,'fit_intercept':clf_fit_intercept} 
        globals()['clf%s' % use_algorithm_name + treatment_id_name] = GridSearchCV(use_algorithm, parameters, cv=5).fit(X_train, y_train)
    elif use_algorithm_name == 'ADA':
        clf_learning_rate=[0.1, 0.5, 1.0, 10.0]
        clf_n_estimators=[10, 50]
        parameters = {'learning_rate':clf_learning_rate, 'n_estimators':clf_n_estimators}
        globals()['clf%s' % use_algorithm_name + treatment_id_name] = GridSearchCV(use_algorithm, parameters, cv=5).fit(X_train, y_train)
    elif use_algorithm_name =='RF':
        n_estimators_values = [10, 50, 100, 500]
        #min_impurity_decrease_values = [0., 0.005, 0.00025]
        max_features_params = [0.1, 0.25, 0.5, 0.75, 'sqrt', 'log2', None]
        criterion_params = ['gini', 'entropy']
        parameters = {'criterion':criterion_params, 'max_features':max_features_params,'n_estimators':n_estimators_values}
        globals()['clf%s' % use_algorithm_name + treatment_id_name] = GridSearchCV(use_algorithm, parameters,error_score=0.0, cv=5).fit(X_train, y_train)
    elif use_algorithm_name == 'DT':
        criterion_params = ['gini','entropy']
        max_features_params = ['auto','sqrt','log2'] 
        parameters = {'criterion':criterion_params, 'max_features':max_features_params}
        globals()['clf%s' % use_algorithm_name + treatment_id_name] = GridSearchCV(use_algorithm, parameters,error_score=0.0, cv=5).fit(X_train, y_train)
    elif use_algorithm_name=='NN':
        learning_rate=['constant', 'invscaling', 'adaptive']
        activation = ['identity', 'logistic', 'tanh', 'relu']
        learningrate_params = [0.01, 0.05, 0.1, 0.5, 1.0] 
        parameters = {'learning_rate':learning_rate,'activation':activation,'learning_rate_init':learningrate_params}
        globals()['clf%s' % use_algorithm_name + treatment_id_name] = GridSearchCV(use_algorithm, parameters, cv=5).fit(X_train, y_train)
    elif use_algorithm_name=='SGD':
        fit_intercept=[True, False]
        l1_ratio=[0, 0.15, 1.0]
        loss_params=['log','modified_huber']
        parameters = {'fit_intercept':fit_intercept,'l1_ratio':l1_ratio,'loss':loss_params}
        globals()['clf%s' % use_algorithm_name + treatment_id_name] = GridSearchCV(use_algorithm, parameters, cv=5).fit(X_train, y_train)
    elif use_algorithm_name=='SVC':
        print(treatment_id_name)
        kernel=['sigmoid','rbf']
        parameters = {'kernel':kernel}
        globals()['clf%s' % use_algorithm_name + treatment_id_name] = GridSearchCV(use_algorithm, parameters, cv=5).fit(X_train, y_train)
    elif use_algorithm_name=='KNN':
        metric_params = ['minkowski','euclidean','manhattan'] 
        weights_params = ['uniform','distance'] 
        numNeighbors_params  = [5, 6, 7, 8, 9]
        parameters = {'metric':metric_params,'weights':weights_params,'n_neighbors':numNeighbors_params}
        globals()['clf%s' % use_algorithm_name + treatment_id_name] = GridSearchCV(use_algorithm, parameters, cv=5).fit(X_train, y_train)
    



In [ ]:
#find parameter id for saving the results of gridsearch
def find_parameter_id_f (algorithm_name,parameter_name):
    cur=con.cursor()
    cur.execute('SELECT ID FROM HFT_PARAMETERS_T WHERE HFT_ALGORITHM_T_NAME like:algorithm_name AND NAME like:parameter_name',algorithm_name=algorithm_name,parameter_name=parameter_name)
    parameter_id=cur.fetchone()[0]
    ##print (cur.fetchone())
    return parameter_id

In [ ]:
#insert tuned parameters per model into database
def insert_into_model_parameters_f (model_id,treatment_id,parameter_id,value):
    cur=con.cursor()
    sql='INSERT INTO HFT_MODEL_PARAMETERS_T(HFT_MODEL_T_ID,HFT_PARAMETERS_T_ID,TREATMENT_ID,VALUE) VALUES(:1,:2,:3,:4)'
    cur.execute(sql,(model_id,parameter_id,treatment_id,value))
    con.commit()

In [ ]:
# select treatment id's for looping through all models etc...
cur = con.cursor()
cur.execute('select distinct treatment_id from hft_sum_steps_v where research_group in (1,2) and weekday not in (5,6)')
#dataframe
df_treatment_id=pd.DataFrame(cur.fetchall(),columns=['treatment_id'])
cur.close()

In [ ]:
#select data from measurements
#added case to select only the weeks where coaching took place research group 1 started in week 5 with coaching, research group 2 started in week 16 with coaching
cur = con.cursor()
for i in df_treatment_id['treatment_id']:
    treatment_id=int(i)
    cur.execute('select id,treatment_id,year,week,weekday,hour,sum_steps, sum_steps_hour,daily_steps from hft_sum_steps_v where \
             treatment_id=:treatment_id and hour in (7,8,9,10,11,12,13,14,15,16,17,18) \
             and weekday not in (5,6) and year=2015 and (case when research_group=2 and week>15 then 1 when research_group=1 and week>4 then 1 else 0 end = 1)\
             order by year,week,weekday,hour',treatment_id=treatment_id)     
    globals()['df%s' % treatment_id]= pd.DataFrame(cur.fetchall(),columns=['id','treatment_id','year','week','weekday','hour','sum_steps','sum_steps_hour','daily_steps'])
#cur.close()

In [ ]:
#select avg steps per day 
#cur = con.cursor()
for i in df_treatment_id['treatment_id']:
    treatment_id=int(i)
    cur.execute( 'select treatment_id,avg(sum_steps_hour) avg_daily_steps from hft_sum_steps_v where treatment_id=:treatment_id\
                  and year=2015 and hour=18 and weekday not in (5,6) and (case when research_group=2 and week>15 then 1 when research_group=1 and week>4 then 1 else 0 end = 1)\
                  group by treatment_id',treatment_id=treatment_id)
    globals()['df_threshold%s' % treatment_id]= pd.DataFrame(cur.fetchall(),columns=['treatment_id','avg_daily_steps'])
#cur.close()

In [ ]:
#set threshold
for i in df_treatment_id['treatment_id']:
    treatment_id=int(i)
    globals()['threshold%s' % treatment_id] = globals()['df_threshold%s' % treatment_id]['avg_daily_steps']
    #print(globals()['df_threshold%s' % treatment_id].head())

In [ ]:
#use numpy vectorize to use function with more arguments and a dataframe determine if the threshold is met
for i in df_treatment_id['treatment_id']:
    treatment_id=int(i)
    x=globals()['df%s' % treatment_id]['daily_steps']
    y=globals()['df_threshold%s' % treatment_id]['avg_daily_steps']
    globals()['df%s' % treatment_id]['dailysteps_cat']=np.vectorize(daily_steps_cat_f)(x, y)
    #print(y)

In [ ]:
#split df 
from sklearn.model_selection import train_test_split
for i in df_treatment_id['treatment_id']:
    treatment_id=int(i)
    X= globals()['df%s' % treatment_id].iloc[:, 5:8].values
    y= globals()['df%s' % treatment_id].iloc[:, 9].values
    #normalize for NN,SVC,SGD and KNN
    min_max_scaler = preprocessing.MinMaxScaler()
    np_scaled = min_max_scaler.fit_transform(X)
    X_s = pd.DataFrame(np_scaled)
    
    globals()['X_train%s' % treatment_id],globals()['X_test%s' % treatment_id],globals()['y_train%s' % treatment_id],globals()['y_test%s' % treatment_id]\
    =train_test_split(X,y, test_size=0.3, random_state=10)    
    
    globals()['X_train_s%s' % treatment_id],globals()['X_test_s%s' % treatment_id],globals()['y_train%s' % treatment_id],globals()['y_test%s' % treatment_id]\
    =train_test_split(X,y, test_size=0.3, random_state=10)    

In [ ]:
#built models for different algorithms
#import warnings
#with warnings.catch_warnings():
    #warnings.simplefilter('ignore')
for i in df_treatment_id['treatment_id']:
    treatment_id=int(i)
    if all_same(globals()['y_train%s' % treatment_id]):
        pass
    else:
        fit_model(treatment_id,'ADA',AdaBoostClassifier(),globals()['X_train%s' % treatment_id],globals()['y_train%s' % treatment_id])
        fit_model(treatment_id,'DT',DecisionTreeClassifier(),globals()['X_train%s' % treatment_id],globals()['y_train%s' % treatment_id])
        fit_model(treatment_id,'KNN',KNeighborsClassifier(),globals()['X_train_s%s' % treatment_id],globals()['y_train%s' % treatment_id])
        fit_model(treatment_id,'LR',LogisticRegression(),globals()['X_train%s' % treatment_id],globals()['y_train%s' % treatment_id])
        fit_model(treatment_id,'NN',MLPClassifier(),globals()['X_train_s%s' % treatment_id],globals()['y_train%s' % treatment_id])
        fit_model(treatment_id,'RF',RandomForestClassifier(),globals()['X_train%s' % treatment_id],globals()['y_train%s' % treatment_id])
        fit_model(treatment_id,'SGD', SGDClassifier(),globals()['X_train_s%s' % treatment_id],globals()['y_train%s' % treatment_id])
        fit_model(treatment_id,'SVC',SVC(probability=True),globals()['X_train_s%s' % treatment_id],globals()['y_train%s' % treatment_id])

In [ ]:
#Prediction for different models
for i in df_treatment_id['treatment_id']:
    treatment_id=int(i)
    if all_same(globals()['y_train%s' % treatment_id]):
        pass
    else:
        
        globals()['X%s' % treatment_id]= globals()['df%s' % treatment_id].iloc[:, 5:8].values
        globals()['y%s' % treatment_id]= globals()['df%s' % treatment_id].iloc[:,9].values
        
        #scale for NN,SVC,SGD and KNN
        min_max_scaler = preprocessing.MinMaxScaler()
        np_scaled = min_max_scaler.fit_transform(globals()['X%s' % treatment_id])
        globals()['X_s%s' % treatment_id] = pd.DataFrame(np_scaled)
        
        
        globals()['y_predADA%s' % treatment_id] = globals()['clfADA%s' % treatment_id].predict(globals()['X%s' % treatment_id])
        globals()['y_predDT%s' % treatment_id] = globals()['clfDT%s' % treatment_id].predict(globals()['X%s' % treatment_id])
        globals()['y_predKNN%s' % treatment_id] = globals()['clfKNN%s' % treatment_id].predict(globals()['X_s%s' % treatment_id])
        globals()['y_predLR%s' % treatment_id] = globals()['clfLR%s' % treatment_id].predict(globals()['X%s' % treatment_id])
        globals()['y_predNN%s' % treatment_id] = globals()['clfNN%s' % treatment_id].predict(globals()['X_s%s' % treatment_id])
        globals()['y_predRF%s' % treatment_id] = globals()['clfRF%s' % treatment_id].predict(globals()['X%s' % treatment_id])
        globals()['y_predSGD%s' % treatment_id] = globals()['clfSGD%s' % treatment_id].predict(globals()['X_s%s' % treatment_id])
        globals()['y_predSVC%s' % treatment_id] = globals()['clfSVC%s' % treatment_id].predict(globals()['X_s%s' % treatment_id])     
          
   

In [ ]:
#Save all models on file system and the desciption and the metrics in the database
dest = pickle_destination_f ('app','pkl_objects')
for i in df_treatment_id['treatment_id']:
    treatment_id=str(i)
    if all_same(globals()['y_train%s' % treatment_id]):
        pass
    else:
        algorithm_name='LR'
        pickle_model=str((treatment_id)+'_'+algorithm_name+'_'+'model.pkl')
        save_model_f(dest,treatment_id,algorithm_name)
        save_model_database_f(treatment_id,pickle_model,algorithm_name,dest)

        algorithm_name='ADA'
        pickle_model=str((treatment_id)+'_'+algorithm_name+'_'+'model.pkl')
        save_model_f(dest,treatment_id,algorithm_name)
        save_model_database_f(treatment_id,pickle_model,algorithm_name,dest)
              
        algorithm_name='RF'
        pickle_model=str((treatment_id)+'_'+algorithm_name+'_'+'model.pkl')
        save_model_f(dest,treatment_id,algorithm_name)
        save_model_database_f(treatment_id,pickle_model,algorithm_name,dest)
        
              
        algorithm_name='DT'
        pickle_model=str((treatment_id)+'_'+algorithm_name+'_'+'model.pkl')
        save_model_f(dest,treatment_id,algorithm_name)
        save_model_database_f(treatment_id,pickle_model,algorithm_name,dest)
        
        algorithm_name='KNN'
        pickle_model=str((treatment_id)+'_'+algorithm_name+'_'+'model.pkl')
        save_model_f(dest,treatment_id,algorithm_name)
        save_model_database_f(treatment_id,pickle_model,algorithm_name,dest)
        
        algorithm_name='NN'
        pickle_model=str((treatment_id)+'_'+algorithm_name+'_'+'model.pkl')
        save_model_f(dest,treatment_id,algorithm_name)
        save_model_database_f(treatment_id,pickle_model,algorithm_name,dest)
                
        algorithm_name='SGD'
        pickle_model=str((treatment_id)+'_'+algorithm_name+'_'+'model.pkl')
        save_model_f(dest,treatment_id,algorithm_name)
        save_model_database_f(treatment_id,pickle_model,algorithm_name,dest)
        
        algorithm_name='SVC'
        pickle_model=str((treatment_id)+'_'+algorithm_name+'_'+'model.pkl')
        save_model_f(dest,treatment_id,algorithm_name)
        save_model_database_f(treatment_id,pickle_model,algorithm_name,dest)
    
